In [1]:
from pymongo import MongoClient
import pprint

In [2]:
client = MongoClient()

In [3]:
db = client.radio

In [4]:
materials = db.material

In [6]:
variableSkelett = {
    "id": "S8",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["frisch"],
        ["alt"]
    ],
    "kind": "oc",
    "value": None,
    "values": ["frisch", "alt"]
}

In [7]:
variablePneumonie0 = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", False],
        ["ML", False]
    ]
}

In [8]:
variablePneumonieOL = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", True],
        ["UL", False],
        ["ML", False]
    ]
}

In [9]:
variablePneumonieUL = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", True],
        ["ML", False]
    ]
}

In [10]:
variablePneumonieML = {
    "id": "Pn2",
    "textBefore": "",
    "textAfter": "",
    "keys": [
        ["OL", "Oberfeld", "Oberlappen"],
        ["UL", "Unterfeld", "Unterlappen"],
        ["ML", "Mittelfeld", "Mittellappen"]
    ],
    "kind": "mc",
    "values": [
        ["OL", False],
        ["UL", False],
        ["ML", True]
    ]
}

In [11]:
oids = []
for material in materials.find({"judged": True}):
    oid = material.get("_id")
    oids.append(oid)

In [12]:
for oid in oids:
    material = materials.find_one({"_id": oid})

    # get variables of selectable 'Fraktur' in category 'Skelett'
    skellyVariables = material.get("deepDocTemplate").get("parts")[23].get("selectables")[11].get("variables")
    # if amount of variables is 2 or less, then no variable has been added yet (as of old template). Only then to be added to variables
    if len(skellyVariables) < 3:
        skellyVariables.append(variableSkelett)

    # get variables of value 'Pneumonie' in first group of category 'Lunge'
    pneumoVariables = material.get("deepDocTemplate").get("parts")[16].get("selectables")[0].get("options")[1].get("variables")

    # only change the variable if the underlying variable is still oc, otherwise no changes required
    if pneumoVariables[2].get("kind") == "oc":
        # check value of variables with index 2 ('OL'/'UL'/ML') and replace said variable based on value
        if pneumoVariables[2].get("value") == 'UL':
            pneumoVariables[2] = variablePneumonieUL
        elif pneumoVariables[2].get("value") == 'ML':
            pneumoVariables[2] = variablePneumonieML
        elif pneumoVariables[2].get("value") == 'OL':
            pneumoVariables[2] = variablePneumonieOL
        else:
            pneumoVariables[2] = variablePneumonie0

    materials.replace_one({"_id": oid}, material)

In [38]:
oid = oids[1]
material = materials.find_one({"_id": oid})
skellyVariables = material.get("deepDocTemplate").get("parts")[23].get("selectables")[11].get("variables")
if len(skellyVariables) < 3:
    skellyVariables.append(variableSkelett)

pneumoVariable = material.get("deepDocTemplate").get("parts")[16].get("selectables")[0].get("options")[1].get("variables")
pprint.pprint(material.get("deepDocTemplate").get("parts")[23])

{'kind': 'category',
 'name': 'Skelett',
 'optional': True,
 'selectables': [{'exclusions': ['Rest'],
                  'keys': ['keine Degenerationen',
                           'keine Degeneration',
                           'Skelett unauffällig'],
                  'kind': 'box',
                  'name': 'Skelett unauffällig',
                  'normal': True,
                  'text': 'Keine relevanten degenerativen Veränderungen.',
                  'value': True,
                  'variables': []},
                 {'enumeration': 'SZ0',
                  'exclusions': ['Skelett unauffällig'],
                  'keys': ['Gefügestörung'],
                  'kind': 'box',
                  'name': 'Gefügestörung',
                  'normal': False,
                  'text': 'des Gefüges %S8% [auf Höhe %S9%]',
                  'value': False,
                  'variables': [{'id': 'S8',
                                 'keys': [['gering'], ['mäßig'], ['deutlich']],
             

### Was zu tun ist:
In Aufnahmetyp: Thorax im Liegen --> Thorax a.p. umbenennen
--> Name und Keys, sonst nix
Index Aufnahmetyp: 0
Index der Gruppe bei Sel: 0
Index Thorax im Liegen in Gruppe: 2
Wichtig: Value der Gruppe prüfen und auch umbenennen, wenn nötig!

In Fremdkörper:
Andere Fremdkörper 2 hinzufügen
--> einfach nur neues Selectable appenden, ez pz
Index Instrumentierung: 2
Einfach an Selectables appenden

In [7]:
fremdkoerper2 = {
    "kind": "box",
    "name": "Andere Fremdkörper 2",
    "value": False,
    "normal": False,
    "exclusions": ["Keine"],
    "variables": [
        {
            "id": "I42",
            "textBefore": "",
            "textAfter": "",
            "keys": [
                ["medizinisch", "Kabel"],
                ["nicht medizinisch", "Metallsplitter"]
            ],
            "kind": "oc",
            "value": None,
            "values": ["medizinisch (Kabel etc.)", "nicht medizinisch (Metallsplitter etc.)"]
        },
        {
            "id": "I43",
            "textBefore": "",
            "textAfter": "",
            "keys": [
                ["extrakorporal"],
                ["intrakorporal"]
            ],
            "kind": "oc",
            "value": None,
            "values": ["extrakorporal", "intrakorporal"]
        }
    ],
    "keys": ["Andere Fremdkörper 2", "Anderer Fremdkörper 2", "Andere Fremdkörper Zwei"],
    "text": "Weitere Fremdkörper [%I42%][, %I43%]"
}

In [6]:
oids = []
for material in materials.find({"judged": True}):
    oid = material.get("_id")
    oids.append(oid)

In [11]:
for oid in oids:
    material = materials.find_one({"_id": oid})

    # get selectable "Thorax im Liegen" from parts
    thoraxLiegen = material.get("deepDocTemplate").get("parts")[0].get("selectables")[0].get("options")[2]
    if thoraxLiegen["name"] == "Thorax im Liegen":
        thoraxLiegen["name"] = "Thorax a.p."
        thoraxLiegen["keys"] = ["Thorax a.p., Thorax ap"]

    # get Instrumentierung
    instrumentierungSel = material["deepDocTemplate"]["parts"][2]["selectables"]
    if len(instrumentierungSel) <= 23:
        instrumentierungSel.append(fremdkoerper2)

    materials.replace_one({"_id": oid}, material)